# Gesture Recognition - Model 8
#### Conv3D Model with comprehensive preprocessing (using OpenCV library - Resize and Data augmetation with Affine transformation, Horizontal Flip, Cropping and Resizing,  Usage of Optimizer, ReduceLROnPlateau to reduce learning rate, and  experimenting with various sizes of batch sizes and Epochs)
#### Removed few dropouts in this Model to avoid underfitting
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import datetime
import os
import cv2

Set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. We Keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # lenght of the video frames
nb_channel = 3  # number of channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [6]:
# Helper function to initialize all the batch image data and labels
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [7]:
def process_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = initialize_batch_data(batch_size)
    
    # For building an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = initialize_batch_data(batch_size)
    
    # For building an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = initialize_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generating a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [8]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)            
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model 8 - Final Selected Model

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [11]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 91s 1s/step - loss: 1.7059 - categorical_accuracy: 0.2905 - val_loss: 1.1785 - val_categorical_accuracy: 0.6100

Epoch 00001: saving model to model_init_2020-03-3011_11_51.849206/model-00001-1.70906-0.29010-1.17850-0.61000.h5
Epoch 2/20
67/67 [==============================] - 26s 394ms/step - loss: 1.3765 - categorical_accuracy: 0.4113 - val_loss: 1.4186 - val_categorical_accuracy: 0.5200

Epoch 00002: saving model to model_init_2020-03-3011_11_51.849206/model-00002-1.37651-0.41128-1.41858-0.52000.h5
Epoch 3/20
67/67 [==============================] - 28s 419ms/step - loss: 1.4036 - categorical_accuracy: 0.4245 - val_loss: 1.4455 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-03-3011_11_51.849206/model-00003-1.40357-0.42454-1.44550-0.40000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate

In [16]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 16s 476ms/step - loss: 0.4073 - categorical_accuracy: 0.8791 - val_loss: 0.5723 - val_categorical_accuracy: 0.7800

Epoch 00001: saving model to model_init_2020-03-3011_11_51.849206/model-00001-0.40728-0.87908-0.57228-0.78000.h5
Epoch 2/20
34/34 [==============================] - 14s 422ms/step - loss: 0.3450 - categorical_accuracy: 0.9052 - val_loss: 0.5354 - val_categorical_accuracy: 0.8400

Epoch 00002: saving model to model_init_2020-03-3011_11_51.849206/model-00002-0.34496-0.90523-0.53537-0.84000.h5
Epoch 3/20
34/34 [==============================] - 14s 399ms/step - loss: 0.3814 - categorical_accuracy: 0.8399 - val_loss: 0.3066 - val_categorical_accuracy: 0.8400

Epoch 00003: saving model to model_init_2020-03-3011_11_51.849206/model-00003-0.38140-0.83987-0.30662-0.84000.h5
Epoch 4/20
34/34 [==============================] - 13s 379ms/step - loss: 0.4034 - categorical_accuracy: 0.8562 - val_loss: 0.3469 - val_categorical_accurac

In [17]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 10s 436ms/step - loss: 0.3915 - categorical_accuracy: 0.8454 - val_loss: 0.5423 - val_categorical_accuracy: 0.7750

Epoch 00001: saving model to model_init_2020-03-3011_11_51.849206/model-00001-0.39154-0.84541-0.54230-0.77500.h5
Epoch 2/30
23/23 [==============================] - 10s 427ms/step - loss: 0.3325 - categorical_accuracy: 0.8647 - val_loss: 0.2863 - val_categorical_accuracy: 0.9000

Epoch 00002: saving model to model_init_2020-03-3011_11_51.849206/model-00002-0.33246-0.86473-0.28630-0.90000.h5
Epoch 3/30
23/23 [==============================] - 9s 408ms/step - loss: 0.5486 - categorical_accuracy: 0.7826 - val_loss: 0.4864 - val_categorical_accuracy: 0.8250

Epoch 00003: saving model to model_init_2020-03-3011_11_51.849206/model-00003-0.54858-0.78261-0.48640-0.82500.h5
Epoch 4/30
23/23 [==============================] - 10s 428ms/step - loss: 0.3911 - categorical_accuracy: 0.8599 - val_loss: 0.3238 - val_categorical_accuracy

23/23 [==============================] - 9s 408ms/step - loss: 0.3964 - categorical_accuracy: 0.8502 - val_loss: 0.4596 - val_categorical_accuracy: 0.8250

Epoch 00027: saving model to model_init_2020-03-3011_11_51.849206/model-00027-0.39640-0.85024-0.45965-0.82500.h5
Epoch 28/30
23/23 [==============================] - 9s 405ms/step - loss: 0.2924 - categorical_accuracy: 0.9034 - val_loss: 0.4144 - val_categorical_accuracy: 0.8000

Epoch 00028: saving model to model_init_2020-03-3011_11_51.849206/model-00028-0.29240-0.90338-0.41437-0.80000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 5.960464760645934e-11.
Epoch 29/30
23/23 [==============================] - 9s 396ms/step - loss: 0.2819 - categorical_accuracy: 0.8937 - val_loss: 0.3245 - val_categorical_accuracy: 0.9000

Epoch 00029: saving model to model_init_2020-03-3011_11_51.849206/model-00029-0.28193-0.89372-0.32445-0.90000.h5
Epoch 30/30
23/23 [==============================] - 9s 412ms/step - loss: 0.4011 - cate

In [18]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 7s 473ms/step - loss: 0.3158 - categorical_accuracy: 0.8492 - val_loss: 0.4534 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2020-03-3011_11_51.849206/model-00001-0.31582-0.84921-0.45341-0.85000.h5
Epoch 2/30
14/14 [==============================] - 6s 431ms/step - loss: 0.3223 - categorical_accuracy: 0.9048 - val_loss: 0.4573 - val_categorical_accuracy: 0.9000

Epoch 00002: saving model to model_init_2020-03-3011_11_51.849206/model-00002-0.32229-0.90476-0.45735-0.90000.h5
Epoch 3/30
14/14 [==============================] - 6s 408ms/step - loss: 0.3065 - categorical_accuracy: 0.8730 - val_loss: 0.3348 - val_categorical_accuracy: 0.8500

Epoch 00003: saving model to model_init_2020-03-3011_11_51.849206/model-00003-0.30649-0.87302-0.33475-0.85000.h5
Epoch 4/30
14/14 [==============================] - 5s 377ms/step - loss: 0.2944 - categorical_accuracy: 0.8810 - val_loss: 0.4783 - val_categorical_accuracy: 0

In [19]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 4s 500ms/step - loss: 0.0984 - categorical_accuracy: 0.9861 - val_loss: 0.5750 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2020-03-3011_11_51.849206/model-00001-0.09838-0.98611-0.57498-0.80000.h5
Epoch 2/30
8/8 [==============================] - 4s 472ms/step - loss: 0.3893 - categorical_accuracy: 0.8611 - val_loss: 0.1421 - val_categorical_accuracy: 1.0000

Epoch 00002: saving model to model_init_2020-03-3011_11_51.849206/model-00002-0.38930-0.86111-0.14213-1.00000.h5
Epoch 3/30
8/8 [==============================] - 4s 474ms/step - loss: 0.3378 - categorical_accuracy: 0.8889 - val_loss: 0.6338 - val_categorical_accuracy: 0.6000

Epoch 00003: saving model to model_init_2020-03-3011_11_51.849206/model-00003-0.33780-0.88889-0.63376-0.60000.h5
Epoch 4/30
8/8 [==============================] - 3s 427ms/step - loss: 0.2845 - categorical_accuracy: 0.9167 - val_loss: 0.2779 - val_categorical_accuracy: 0.9500

E

In [20]:
batch_size = 90
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 4s 514ms/step - loss: 0.4342 - categorical_accuracy: 0.8333 - val_loss: 0.2982 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2020-03-3011_11_51.849206/model-00001-0.43420-0.83333-0.29819-0.85000.h5
Epoch 2/20
8/8 [==============================] - 3s 398ms/step - loss: 0.2076 - categorical_accuracy: 0.9444 - val_loss: 0.3932 - val_categorical_accuracy: 0.8500

Epoch 00002: saving model to model_init_2020-03-3011_11_51.849206/model-00002-0.20757-0.94444-0.39322-0.85000.h5
Epoch 3/20
8/8 [==============================] - 3s 383ms/step - loss: 0.3444 - categorical_accuracy: 0.8750 - val_loss: 0.2612 - val_categorical_accuracy: 0.9000

Epoch 00003: saving model to model_init_2020-03-3011_11_51.849206/model-00003-0.34440-0.87500-0.26118-0.90000.h5
Epoch 4/20
8/8 [==============================] - 4s 517ms/step - loss: 0.3417 - categorical_accuracy: 0.8889 - val_loss: 0.6870 - val_categorical_accuracy: 0.7000

E